In [1]:
# A l'issue du NB 2, notre dataset compte 247 variables. Toutes ne peuvent pas être traitées et présentées en DataViz.
# Objectif de ce notebook 4 : extraire les variables les plus significatives que nous présenterons en Dataviz ().
# Méthode suggérée par Alban : faire un remplacement de Nan par la médiane pour toutes les variables numériques 
# et par le mode pour toutes les variables catégorielles.
# Puis après prétraitement (dichotomisation, standardisation...), lancer des modèles de classification 
# et extraire les variables les plus importantes.
# A l'issue de ce notebook, on note que les modèles ne sont par performants.
# On décide de retravailler chaque variable une par une, en même temps que nous faisons la DataViz (voir NB suivant)


In [2]:
import pandas as pd
import numpy as np


In [3]:
df_selec=pd.read_csv("df_selec.csv", index_col="idmutation")

FileNotFoundError: ignored

In [ ]:
df_selec.head()

In [ ]:
df_selec.info()

In [ ]:
# Création du train test split

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
# Variables explicatives  / variable cible

X=df_selec.drop("y", axis=1)
y=df_selec["y"]

X.shape, y.shape

In [ ]:
numeric_columns=X.select_dtypes(include=np.number).columns.tolist()
numeric_columns

In [ ]:
object_columns=X.select_dtypes(include='object').columns.tolist()
object_columns

In [ ]:
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.25, random_state=10)
X_train.shape, X_test.shape

In [ ]:
#Traitement des nan

for col in numeric_columns:
    X_train[col].fillna(X_train[col].median(), inplace=True)
    X_test[col].fillna(X_train[col].median(), inplace=True)

for col in object_columns:
    X_train[col].fillna(X_train[col].mode()[0], inplace=True)
    X_test[col].fillna(X_train[col].mode()[0], inplace=True)

In [ ]:
X_train.isna().sum()

In [ ]:
X_test.isna().sum()

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train=pd.get_dummies(X_train)


In [ ]:
X_train.shape

In [ ]:
X_test=pd.get_dummies(X_test)

In [ ]:
X_test.shape

In [ ]:
# Différence de nombre de colonnes entre X_train et X_test => Pourquoi ? 
# J'ai gardé ces lignes de codes après retraitement des variables pour expliquer raisonnement

In [ ]:
X_train_list=X_train.columns.tolist()

In [ ]:
X_test_list=X_test.columns.tolist()

In [ ]:
difference_train = set(X_train_list).difference(set(X_test_list))
difference_test = set(X_test_list).difference(set(X_train_list))
list_difference = list(difference_train.union(difference_test))
print(list_difference)



In [ ]:
# Normaliser les variables quantitatives

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])


In [ ]:
#LOGISTIC REGRESSION

from sklearn.linear_model import LogisticRegression

clf_lr=LogisticRegression()
clf_lr.fit(X_train, y_train)

In [ ]:
y_pred_lr=clf_lr.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred_lr)
cm

pd.crosstab(y_test, y_pred_lr)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_lr))

In [ ]:
# DECISION TREE

from sklearn.tree import DecisionTreeClassifier
clf_dt=DecisionTreeClassifier()


In [ ]:

clf_dt.fit(X_train, y_train)
y_pred_dt=clf_dt.predict(X_test)

In [ ]:
cm=pd.crosstab(y_test, y_pred_dt, rownames=["Classe réelle"], colnames=["Classe prédite"])
cm

In [ ]:
# Affichage des meilleurs paramètres
feats = {}
for feature, importance in zip(X.columns, clf_dt.feature_importances_):
    feats[feature] = importance 
    
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Importance'})
importances.sort_values(by='Importance', ascending=False).head(8)

In [ ]:
clf_dt.score(X_test,y_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_dt))

In [ ]:
# Utilisation d'un RandomClassifier = 

from sklearn.ensemble import RandomForestClassifier
clf_rf=RandomForestClassifier()
clf_rf.fit(X_train, y_train)
y_pred_rf=clf_rf.predict(X_test)
pd.crosstab(y_test, y_pred_rf, rownames=["Classe réelle"], colnames=["Classe prédite"])


In [ ]:
print(classification_report(y_test, y_pred_rf))

In [ ]:
feats = {}
for feature, importance in zip(X.columns, clf_rf.feature_importances_):
    feats[feature] = importance 
    
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Importance'})
importances.sort_values(by='Importance', ascending=False).head(8)